Prerequisite installs for Project 2 - Quantum teleportation (Simulation)

In [ ]:
%pip install qiskit
%pip install qiskit[visualization]
%pip install qiskit-aer

Project 2 - Quantum teleportation (Simulation)

In [ ]:
from math                 import sqrt
from qiskit               import QuantumRegister,   \
                                 ClassicalRegister, \
                                 QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_aer           import AerSimulator


# Creating registers
q = QuantumRegister(name='q', size=3)
c = ClassicalRegister(name='c', size=3)
teleport = QuantumCircuit(q, c)

# Step 0 - Instantiate qubit 0 in the state to be teleported
teleport.initialize([1 / sqrt(2), 1j / sqrt(2)], q[0]) # Initialize q[0] (belonging to Alice) to |ϕ⟩ = (|0⟩ + i|1⟩) / √2
teleport.barrier()

# Step 1 - Make the shared entangled state in between qubit 1 and qubit 2
teleport.h(q[1])        # Apply Hadamard gate to q[1]
teleport.cx(q[1], q[2]) # Apply CNOT gate with q[1] as control and q[2] as target
teleport.barrier()

# Step 2 & 3: Alice applies a series of gates on qubit 0 and qubit 1
teleport.cx(q[0], q[1]) # Apply CNOT gate with q[0] as control and q[1] as target
teleport.h(q[0])        # Apply Hadamard gate on q[0]
teleport.barrier()

# Step 4 - Alice measures qubit 0 and qubit 1
teleport.measure(q[0], c[0]) # Measure q[0] into c[0]
teleport.measure(q[1], c[1]) # Measure q[1] into c[1]
teleport.barrier()

# Step 5 - Bob applies a unitary transformation on his qubit based on the outcome of Alice's measurements
teleport.x(q[2]).c_if(c[1], 1) # Apply X gate on q[2] if c[1] == 1
teleport.z(q[2]).c_if(c[0], 1) # Apply Z gate on q[2] if c[0] == 1
teleport.barrier()

# Visualizing your teleportation circuit
teleport.draw()

# How can Bob quickly check if his qubit is actually in state |ϕ⟩=
# Implement your solution...
# Step 6 - Bob checks if he got the right state
teleport.measure(q[2], c[2]) # Measure q[2] into c[2]

# And then run a simulation:
backend = AerSimulator()
shots = 8192
result = backend.run(teleport, shots=shots).result()
counts = result.get_counts()
print(f"The counts for circuit teleport are: {counts}")
plot_histogram(counts)

Prerequisite installs for Project 2 - Quantum teleportation (IBM Quantum Platform)

In [ ]:
%pip install qiskit
%pip install qiskit[visualization]
%pip install qiskit-ibm-runtime

Save account to Qiskit, according to configuration file 'ibm_quantum_config.json'.
To get your IBM Quantum Platform token, go to https://quantum.ibm.com/account and copy your API token.
The 'ibm_quantum_config.json' file should contain something like this:

{
    "channel": "ibm_quantum",
    "token": "yourtoken"
}

In [ ]:
import os
import sys
import json
from   qiskit_ibm_runtime import QiskitRuntimeService


if not os.path.exists("ibm_quantum_config.json"):
    print("Please create a file named 'ibm_quantum_config.json' with your IBM Quantum API token.")
    sys.exit(1)

with open("ibm_quantum_config.json", "r") as config_file:
    config = json.load(config_file)
    print("Configuration loaded successfully.")

QiskitRuntimeService.save_account(
    channel=config["channel"],
    token=config["token"],
    overwrite=True
)

print("Account saved successfully.")

Load account from Qiskit.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService


service = QiskitRuntimeService()
print("Account loaded successfully!")

backends = service.backends(
    filters=lambda b: not b.configuration().simulator and b.status().operational
)
print(f"Available Backends: {backends}")

least_busy_backend = service.least_busy(
    filters=lambda b: not b.configuration().simulator and b.status().operational
)
print(f"Least busy backend: {least_busy_backend.name}")

least_busy_backend_nb_qubits = least_busy_backend.configuration().n_qubits
print(f"Least busy backend number of qubits: {least_busy_backend_nb_qubits}")

Project 2 - Quantum teleportation (IBM Quantum Platform)

In [ ]:
from math                 import sqrt
from qiskit               import QuantumRegister,   \
                                 ClassicalRegister, \
                                 QuantumCircuit,    \
                                 transpile
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime   import Session, \
                                 Sampler

# Creating registers
q = QuantumRegister(name='q', size=3)
c = ClassicalRegister(name='c', size=3)
teleport = QuantumCircuit(q, c)

# Step 0 - Instantiate qubit 0 in the state to be teleported
teleport.initialize([1 / sqrt(2), 1j / sqrt(2)], q[0])  # Initialize q[0] to |ϕ⟩ = (|0⟩ + i|1⟩) / √2
teleport.barrier()

# Step 1 - Make the shared entangled state in between qubit 1 and qubit 2
teleport.h(q[1])        # Apply Hadamard gate to q[1]
teleport.cx(q[1], q[2]) # Apply CNOT gate with q[1] as control and q[2] as target
teleport.barrier()

# Step 2 & 3: Alice applies a series of gates on qubit 0 and qubit 1
teleport.cx(q[0], q[1]) # Apply CNOT gate with q[0] as control and q[1] as target
teleport.h(q[0])        # Apply Hadamard gate on q[0]
teleport.barrier()

# Step 4 - Alice measures qubit 0 and qubit 1
teleport.measure(q[0], c[0])  # Measure q[0] into c[0]
teleport.measure(q[1], c[1])  # Measure q[1] into c[1]
teleport.barrier()

# Step 5 - Bob applies a unitary transformation on his qubit based on the outcome of Alice's measurements
teleport.x(q[2]).c_if(c[1], 1)  # Apply X gate on q[2] if c[1] == 1
teleport.z(q[2]).c_if(c[0], 1)  # Apply Z gate on q[2] if c[0] == 1
teleport.barrier()

# Step 6 - Bob checks if he got the right state
teleport.measure(q[2], c[2])  # Measure q[2] into c[2]

# Transpile the circuit for the previously selected least busy backend
teleport_transpiled = transpile(teleport, least_busy_backend)

# Use Qiskit Runtime with Sampler for the quantum circuit
with Session(backend=least_busy_backend) as session:
    sampler = Sampler(mode=session)

    shots = 8192
    job = sampler.run([teleport_transpiled], shots=shots)
    job_result = job.result()

    # Obtain the counts (measurement results) from the job result
    counts = job_result[0].data.c.get_counts()
    print(f"The counts for circuit teleport are: {counts}")
    plot_histogram(counts)